### requirements for the following codings


In [2]:
### packages required 
!pip install fair-esm 
!pip install torch
!pip install tensorflow
!pip install sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 93 kB 1.3 MB/s 


### peptide embeddings with esm2_t6_8M_UR50D pretrained models
6 layers, 8M parameters, dataset: UR50/D 2021_04, embedding dimension: 320
mode download URL: https://dl.fbaipublicfiles.com/fair-esm/models/esm2_t6_8M_UR50D.pt

In [18]:
def esm_embeddings(peptide_sequence_list):
  # NOTICE: ESM for embeddings is quite RAM usage, if your sequence is too long, 
  #         or you have too many sequences for transformation in a single converting, 
  #         you conputer might automatically kill the job.
  import torch
  import esm
  import collections
  # load the model
  # NOTICE: if the model was not downloaded in your local environment, it will automatically download it.
  model, alphabet = esm.pretrained.esm2_t6_8M_UR50D()
  batch_converter = alphabet.get_batch_converter()
  model.eval()  # disables dropout for deterministic results

  # load the peptide sequence list into the bach_converter
  batch_labels, batch_strs, batch_tokens = batch_converter(peptide_sequence_list)
  batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)
  ## batch tokens are the embedding results of the whole data set

  # Extract per-residue representations (on CPU)
  with torch.no_grad():
      # Here we export the last layer of the EMS model output as the representation of the peptides
      # model'esm2_t6_8M_UR50D' only has 6 layers, and therefore repr_layers parameters is equal to 6
      results = model(batch_tokens, repr_layers=[6], return_contacts=True)  
  token_representations = results["representations"][6]

  # Generate per-sequence representations via averaging
  # NOTE: token 0 is always a beginning-of-sequence token, so the first residue is token 1.
  sequence_representations = []
  for i, tokens_len in enumerate(batch_lens):
      sequence_representations.append(token_representations[i, 1 : tokens_len - 1].mean(0))
  # save dataset
  # sequence_representations is a list and each element is a tensor
  embeddings_results = collections.defaultdict(list)
  for i in range(len(sequence_representations)):
      # tensor can be transformed as numpy sequence_representations[0].numpy() or sequence_representations[0].to_list
      each_seq_rep = sequence_representations[i].tolist()
      for each_element in each_seq_rep:
          embeddings_results[i].append(each_element)
  embeddings_results = pd.DataFrame(embeddings_results).T
  return embeddings_results

### data loading and embeddings (main dataset)

In [4]:
import numpy as np
import pandas as pd

In [44]:
# training dataset loading
dataset = pd.read_excel('AMAP_train_main.xlsx',na_filter = False) # take care the NA sequence problem
sequence_list = dataset['sequence'] 

embeddings_results = pd.DataFrame()
for seq in sequence_list:
    format_seq = [seq,seq] # the setting is just following the input format setting in ESM model, [name,sequence]
    tuple_sequence = tuple(format_seq)
    peptide_sequence_list = []
    peptide_sequence_list.append(tuple_sequence) # build a summarize list variable including all the sequence information
    # employ ESM model for converting and save the converted data in csv format
    one_seq_embeddings = esm_embeddings(peptide_sequence_list)
    embeddings_results= pd.concat([embeddings_results,one_seq_embeddings])

embeddings_results.to_csv('AMAP_train_main_esm2_t6_8M_UR50D_unified_320_dimension.csv')

# loading the y dataset for model development 
y_train = dataset['label']
y_train = np.array(y_train) # transformed as np.array for CNN model

In [64]:
# test dataset loading
dataset = pd.read_excel('AMAP_test_main.xlsx',na_filter = False) # take care the NA sequence problem
sequence_list = dataset['sequence'] 
embeddings_results = pd.DataFrame()
# embedding all the peptide one by one
for seq in sequence_list:
    format_seq = [seq,seq] # the setting is just following the input format setting in ESM model, [name,sequence]
    tuple_sequence = tuple(format_seq)
    peptide_sequence_list = []
    peptide_sequence_list.append(tuple_sequence) # build a summarize list variable including all the sequence information
    # employ ESM model for converting and save the converted data in csv format
    one_seq_embeddings = esm_embeddings(peptide_sequence_list)
    embeddings_results= pd.concat([embeddings_results,one_seq_embeddings])

embeddings_results.to_csv('AMAP_test_main_esm2_t6_8M_UR50D_unified_320_dimension.csv')


# loading the y dataset for model development 
y_test = dataset['label']
y_test = np.array(y_test) # transformed as np.array for CNN model

In [62]:
# assign the dataset 
X_train_data_name = 'AMAP_train_main_esm2_t6_8M_UR50D_unified_320_dimension.csv'
X_train_data = pd.read_csv(X_train_data_name,header=0, index_col = 0,delimiter=',')

X_test_data_name = 'AMAP_test_main_esm2_t6_8M_UR50D_unified_320_dimension.csv'
X_test_data = pd.read_csv(X_test_data_name,header=0, index_col = 0,delimiter=',')

X_train = np.array(X_train_data)
X_test = np.array(X_test_data)

# normalize the X data range
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train) # normalize X to 0-1 range 
X_test = scaler.transform(X_test)

In [65]:
# check the dimension of the dataset before model development
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1819, 320)
(455, 320)
(1819, 1)
(455,)


### data loading and embeddings (alternative dataset)

In [69]:
import numpy as np
import pandas as pd

In [70]:
# training dataset loading
dataset = pd.read_excel('AMAP_train_alternative.xlsx',na_filter = False) # take care the NA sequence problem
sequence_list = dataset['sequence'] 

embeddings_results = pd.DataFrame()
# embedding all the peptide one by one
for seq in sequence_list:
    format_seq = [seq,seq] # the setting is just following the input format setting in ESM model, [name,sequence]
    tuple_sequence = tuple(format_seq)
    peptide_sequence_list = []
    peptide_sequence_list.append(tuple_sequence) # build a summarize list variable including all the sequence information
    # employ ESM model for converting and save the converted data in csv format
    one_seq_embeddings = esm_embeddings(peptide_sequence_list)
    embeddings_results= pd.concat([embeddings_results,one_seq_embeddings])

embeddings_results.to_csv('AMAP_train_alternative_esm2_t6_8M_UR50D_unified_320_dimension.csv')

# loading the y dataset for model development 
y_train = dataset['label']
y_train = np.array(y_train) # transformed as np.array for CNN model

In [71]:
# test dataset loading
dataset = pd.read_excel('AMAP_test_alternative.xlsx',na_filter = False) # take care the NA sequence problem
sequence_list = dataset['sequence'] 

embeddings_results = pd.DataFrame()
# embedding all the peptide one by one
for seq in sequence_list:
    format_seq = [seq,seq] # the setting is just following the input format setting in ESM model, [name,sequence]
    tuple_sequence = tuple(format_seq)
    peptide_sequence_list = []
    peptide_sequence_list.append(tuple_sequence) # build a summarize list variable including all the sequence information
    # employ ESM model for converting and save the converted data in csv format
    one_seq_embeddings = esm_embeddings(peptide_sequence_list)
    embeddings_results= pd.concat([embeddings_results,one_seq_embeddings])

embeddings_results.to_csv('AMAP_test_alternative_esm2_t6_8M_UR50D_unified_320_dimension.csv')

# loading the y dataset for model development 
y_test = dataset['label']
y_test = np.array(y_test) # transformed as np.array for CNN model

In [72]:
# assign the dataset 
X_train_data_name = 'AMAP_train_alternative_esm2_t6_8M_UR50D_unified_320_dimension.csv'
X_train_data = pd.read_csv(X_train_data_name,header=0, index_col = 0,delimiter=',')

X_test_data_name = 'AMAP_test_alternative_esm2_t6_8M_UR50D_unified_320_dimension.csv'
X_test_data = pd.read_csv(X_test_data_name,header=0, index_col = 0,delimiter=',')

X_train = np.array(X_train_data)
X_test = np.array(X_test_data)

# normalize the X data range
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train) # normalize X to 0-1 range 
X_test = scaler.transform(X_test)

In [73]:
# check the dimension of the dataset before model development
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(653, 320)
(163, 320)
(653,)
(163,)


### Model architecture

In [74]:
def ESM_CNN(X_train, y_train, X_test, y_test):
  from keras.layers import Input,InputLayer, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D,Conv1D
  from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, AveragePooling1D, MaxPooling1D
  from keras.models import Sequential,Model
  from keras.optimizers import SGD
  from keras.callbacks import ModelCheckpoint,LearningRateScheduler, EarlyStopping
  import keras
  from keras import backend as K
  inputShape=(320,1)
  input = Input(inputShape)
  x = Conv1D(128,(3),strides = (1),name='layer_conv1',padding='same')(input)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)
  x = MaxPooling1D((2), name='MaxPool1',padding="same")(x)
  x = Dropout(0.15)(x)
  x = Conv1D(32,(3),strides = (1),name='layer_conv2',padding='same')(input)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)
  x = MaxPooling1D((2), name='MaxPool2',padding="same")(x)
  x = Dropout(0.15)(x)
  x = Flatten()(x)
  x = Dense(64,activation = 'relu',name='fc1')(x)
  x = Dropout(0.15)(x)
  x = Dense(2,activation = 'softmax',name='fc2')(x)
  model = Model(inputs = input,outputs = x,name='Predict')
  # define SGD optimizer
  momentum = 0.5
  sgd = SGD(lr=0.01, momentum=momentum, decay=0.0, nesterov=False)
  # compile the model
  model.compile(loss='sparse_categorical_crossentropy',optimizer=sgd, metrics=['accuracy'])
  # learning deccay setting
  import math
  def step_decay(epoch): # gradually decrease the learning rate 
      initial_lrate=0.1
      drop=0.6
      epochs_drop = 3.0
      lrate= initial_lrate * math.pow(drop,    # math.pow base raised to a power
            math.floor((1+epoch)/epochs_drop)) # math.floor Round numbers down to the nearest integer
      return lrate
  lrate = LearningRateScheduler(step_decay)

  # early stop setting
  early_stop = EarlyStopping(monitor='val_accuracy', patience = 40,restore_best_weights = True)

  # summary the callbacks_list
  callbacks_list = [ lrate , early_stop]

  model_history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                            epochs=200,callbacks=callbacks_list,batch_size = 8, verbose=1)
  return model, model_history

### 10-fold cross validation

In [90]:
#Implementing 10-fold cross validation
from sklearn.model_selection import KFold
k = 10 
kf = KFold(n_splits=k, shuffle = True, random_state=1)
X_train = pd.DataFrame(X_train)
y_train = pd.DataFrame(y_train)

# result collection list
ACC_collecton = []
BACC_collecton = []
Sn_collecton = []
Sp_collecton = []
MCC_collecton = []
AUC_collecton = []

for train_index , test_index in kf.split(y_train):
    X_train_CV , X_valid_CV = X_train.iloc[train_index,:],X_train.iloc[test_index,:]
    y_train_CV , y_valid_CV = y_train.iloc[train_index] , y_train.iloc[test_index]
    model, model_history = ESM_CNN(X_train_CV, y_train_CV, X_valid_CV, y_valid_CV)
    # confusion matrix 
    predicted_class= []
    predicted_protability = model.predict(X_valid_CV,batch_size=1)
    for i in range(predicted_protability.shape[0]):
      index = np.where(predicted_protability[i] == np.amax(predicted_protability[i]))[0][0]
      predicted_class.append(index)
    predicted_class = np.array(predicted_class)
    y_true = y_valid_CV    
    from sklearn.metrics import confusion_matrix
    import math
    # np.ravel() return a flatten 1D array
    TP, FP, FN, TN = confusion_matrix(y_true, predicted_class).ravel() # shape [ [True-Positive, False-positive], [False-negative, True-negative] ]
    ACC = (TP+TN)/(TP+TN+FP+FN)
    ACC_collecton.append(ACC)
    Sn_collecton.append(TP/(TP+FN))
    Sp_collecton.append(TN/(TN+FP))
    MCC = (TP*TN-FP*FN)/math.pow(((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)),0.5)
    MCC_collecton.append(MCC)
    BACC_collecton.append(0.5*TP/(TP+FN)+0.5*TN/(TN+FP))
    from sklearn.metrics import roc_auc_score
    AUC = roc_auc_score(y_valid_CV, predicted_protability[:,1])
    AUC_collecton.append(AUC)


Epoch 1/200


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


74/74 [==============================] - 2s 13ms/step - loss: 1.6098 - accuracy: 0.8790 - val_loss: 0.3579 - val_accuracy: 0.8333 - lr: 0.1000
Epoch 2/200
74/74 [==============================] - 1s 15ms/step - loss: 0.1665 - accuracy: 0.9455 - val_loss: 0.3097 - val_accuracy: 0.8636 - lr: 0.1000
Epoch 3/200
74/74 [==============================] - 1s 13ms/step - loss: 0.0987 - accuracy: 0.9642 - val_loss: 0.1854 - val_accuracy: 0.9091 - lr: 0.0600
Epoch 4/200
74/74 [==============================] - 1s 9ms/step - loss: 0.0801 - accuracy: 0.9710 - val_loss: 0.1928 - val_accuracy: 0.9394 - lr: 0.0600
Epoch 5/200
74/74 [==============================] - 1s 8ms/step - loss: 0.0707 - accuracy: 0.9779 - val_loss: 0.2465 - val_accuracy: 0.9091 - lr: 0.0600
Epoch 6/200
74/74 [==============================] - 1s 9ms/step - loss: 0.0473 - accuracy: 0.9847 - val_loss: 0.2469 - val_accuracy: 0.9091 - lr: 0.0360
Epoch 7/200
74/74 [==============================] - 1s 9ms/step - loss: 0.0449 - acc

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


74/74 [==============================] - 2s 13ms/step - loss: 0.8670 - accuracy: 0.9012 - val_loss: 0.5982 - val_accuracy: 0.7727 - lr: 0.1000
Epoch 2/200
74/74 [==============================] - 1s 9ms/step - loss: 0.1289 - accuracy: 0.9455 - val_loss: 0.6094 - val_accuracy: 0.7727 - lr: 0.1000
Epoch 3/200
74/74 [==============================] - 1s 9ms/step - loss: 0.0869 - accuracy: 0.9710 - val_loss: 0.5888 - val_accuracy: 0.7727 - lr: 0.0600
Epoch 4/200
74/74 [==============================] - 1s 9ms/step - loss: 0.0718 - accuracy: 0.9693 - val_loss: 0.5411 - val_accuracy: 0.7879 - lr: 0.0600
Epoch 5/200
74/74 [==============================] - 1s 9ms/step - loss: 0.0595 - accuracy: 0.9847 - val_loss: 0.3931 - val_accuracy: 0.8939 - lr: 0.0600
Epoch 6/200
74/74 [==============================] - 1s 9ms/step - loss: 0.0292 - accuracy: 0.9881 - val_loss: 0.2524 - val_accuracy: 0.9091 - lr: 0.0360
Epoch 7/200
74/74 [==============================] - 1s 9ms/step - loss: 0.0265 - accur

...

74/74 [==============================] - 2s 13ms/step - loss: 1.6665 - accuracy: 0.8605 - val_loss: 0.4005 - val_accuracy: 0.8462 - lr: 0.1000
Epoch 2/200
74/74 [==============================] - 1s 9ms/step - loss: 0.1983 - accuracy: 0.9371 - val_loss: 0.3258 - val_accuracy: 0.8462 - lr: 0.1000
Epoch 3/200
74/74 [==============================] - 1s 9ms/step - loss: 0.1124 - accuracy: 0.9626 - val_loss: 0.2616 - val_accuracy: 0.8769 - lr: 0.0600
Epoch 4/200
74/74 [==============================] - 1s 10ms/step - loss: 0.0984 - accuracy: 0.9711 - val_loss: 0.1681 - val_accuracy: 0.9231 - lr: 0.0600
Epoch 5/200
74/74 [==============================] - 1s 10ms/step - loss: 0.0894 - accuracy: 0.9677 - val_loss: 0.1390 - val_accuracy: 0.9385 - lr: 0.0600
Epoch 6/200
74/74 [==============================] - 1s 9ms/step - loss: 0.0600 - accuracy: 0.9728 - val_loss: 0.1150 - val_accuracy: 0.9538 - lr: 0.0360
Epoch 7/200
74/74 [==============================] - 1s 10ms/step - loss: 0.0562 - ac

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


74/74 [==============================] - 2s 13ms/step - loss: 1.1751 - accuracy: 0.8588 - val_loss: 0.3532 - val_accuracy: 0.8308 - lr: 0.1000
Epoch 2/200
74/74 [==============================] - 1s 10ms/step - loss: 0.1922 - accuracy: 0.9116 - val_loss: 0.3213 - val_accuracy: 0.9077 - lr: 0.1000
Epoch 3/200
74/74 [==============================] - 1s 9ms/step - loss: 0.1374 - accuracy: 0.9456 - val_loss: 0.2222 - val_accuracy: 0.9538 - lr: 0.0600
Epoch 4/200
74/74 [==============================] - 1s 10ms/step - loss: 0.1081 - accuracy: 0.9507 - val_loss: 0.1145 - val_accuracy: 0.9846 - lr: 0.0600
Epoch 5/200
74/74 [==============================] - 1s 9ms/step - loss: 0.0951 - accuracy: 0.9524 - val_loss: 0.0369 - val_accuracy: 1.0000 - lr: 0.0600
Epoch 6/200
74/74 [==============================] - 1s 10ms/step - loss: 0.0673 - accuracy: 0.9694 - val_loss: 0.0091 - val_accuracy: 1.0000 - lr: 0.0360
Epoch 7/200
74/74 [==============================] - 1s 9ms/step - loss: 0.0714 - ac

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


74/74 [==============================] - 2s 12ms/step - loss: 1.1164 - accuracy: 0.8759 - val_loss: 0.3313 - val_accuracy: 0.8615 - lr: 0.1000
Epoch 2/200
74/74 [==============================] - 1s 10ms/step - loss: 0.1361 - accuracy: 0.9524 - val_loss: 0.3515 - val_accuracy: 0.9077 - lr: 0.1000
Epoch 3/200
74/74 [==============================] - 1s 10ms/step - loss: 0.0902 - accuracy: 0.9609 - val_loss: 0.2833 - val_accuracy: 0.9385 - lr: 0.0600
Epoch 4/200
74/74 [==============================] - 1s 10ms/step - loss: 0.0720 - accuracy: 0.9677 - val_loss: 0.1863 - val_accuracy: 0.9231 - lr: 0.0600
Epoch 5/200
74/74 [==============================] - 1s 9ms/step - loss: 0.0534 - accuracy: 0.9762 - val_loss: 0.1458 - val_accuracy: 0.9538 - lr: 0.0600
Epoch 6/200
74/74 [==============================] - 1s 10ms/step - loss: 0.0448 - accuracy: 0.9830 - val_loss: 0.1299 - val_accuracy: 0.9538 - lr: 0.0360
Epoch 7/200
74/74 [==============================] - 1s 9ms/step - loss: 0.0328 - a

In [91]:
from statistics import mean, stdev
print(mean(ACC_collecton),'±',stdev(ACC_collecton))
print(mean(BACC_collecton),'±',stdev(BACC_collecton))
print(mean(Sn_collecton),'±',stdev(Sn_collecton))
print(mean(Sp_collecton),'±',stdev(Sp_collecton))
print(mean(MCC_collecton),'±',stdev(MCC_collecton))
print(mean(AUC_collecton),'±',stdev(AUC_collecton))


0.9679254079254079 ± 0.019494886009330258
0.958523953171012 ± 0.032767742316934384
0.9424664224664224 ± 0.06297559791341326
0.9745814838756015 ± 0.017720334904160737
0.8750757926127642 ± 0.07510612759780215
0.9678133419625546 ± 0.04491376065460708


In [84]:
ACC_collecton

[0.9815950920245399]

### model evaluation in test dataset

In [88]:
# result collection list
ACC_collecton = []
BACC_collecton = []
Sn_collecton = []
Sp_collecton = []
MCC_collecton = []
AUC_collecton = []
model, model_history = ESM_CNN(X_train, y_train, X_test , y_test)
# confusion matrix 
predicted_class= []
predicted_protability = model.predict(X_test,batch_size=1)
for i in range(predicted_protability.shape[0]):
  index = np.where(predicted_protability[i] == np.amax(predicted_protability[i]))[0][0]
  predicted_class.append(index)
predicted_class = np.array(predicted_class)
y_true = y_test    
from sklearn.metrics import confusion_matrix
import math
# np.ravel() return a flatten 1D array
TP, FP, FN, TN = confusion_matrix(y_true, predicted_class).ravel() # shape [ [True-Positive, False-positive], [False-negative, True-negative] ]
ACC = (TP+TN)/(TP+TN+FP+FN)
ACC_collecton.append(ACC)
Sn_collecton.append(TP/(TP+FN))
Sp_collecton.append(TN/(TN+FP))
MCC = (TP*TN-FP*FN)/math.pow(((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)),0.5)
MCC_collecton.append(MCC)
BACC_collecton.append(0.5*TP/(TP+FN)+0.5*TN/(TN+FP))
from sklearn.metrics import roc_auc_score
AUC = roc_auc_score(y_test, predicted_protability[:,1])
AUC_collecton.append(AUC)

Epoch 1/200


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


82/82 [==============================] - 2s 12ms/step - loss: 1.2458 - accuracy: 0.8836 - val_loss: 0.5401 - val_accuracy: 0.8282 - lr: 0.1000
Epoch 2/200
82/82 [==============================] - 1s 10ms/step - loss: 0.1424 - accuracy: 0.9372 - val_loss: 0.4789 - val_accuracy: 0.8528 - lr: 0.1000
Epoch 3/200
82/82 [==============================] - 1s 9ms/step - loss: 0.0968 - accuracy: 0.9648 - val_loss: 0.3865 - val_accuracy: 0.8896 - lr: 0.0600
Epoch 4/200
82/82 [==============================] - 1s 10ms/step - loss: 0.0843 - accuracy: 0.9709 - val_loss: 0.1568 - val_accuracy: 0.9693 - lr: 0.0600
Epoch 5/200
82/82 [==============================] - 1s 10ms/step - loss: 0.0752 - accuracy: 0.9724 - val_loss: 0.1065 - val_accuracy: 0.9632 - lr: 0.0600
Epoch 6/200
82/82 [==============================] - 1s 9ms/step - loss: 0.0637 - accuracy: 0.9786 - val_loss: 0.0781 - val_accuracy: 0.9693 - lr: 0.0360
Epoch 7/200
82/82 [==============================] - 1s 9ms/step - loss: 0.0643 - ac

In [89]:
print(ACC_collecton[0])
print(BACC_collecton[0])
print(Sn_collecton[0])
print(Sp_collecton[0])
print(MCC_collecton[0])
print(AUC_collecton[0])

0.9754601226993865
0.9698203256597417
0.9615384615384616
0.9781021897810219
0.9121428430507218
0.987037037037037


In [79]:
model.save('AMAP_alternative_tensorflow_model',save_format = 'tf') 
!zip -r /content/AMAP_alternative_tensorflow_model.zip /content/AMAP_alternative_tensorflow_model

  adding: content/AMAP_alternative_tensorflow_model/ (stored 0%)
  adding: content/AMAP_alternative_tensorflow_model/variables/ (stored 0%)
  adding: content/AMAP_alternative_tensorflow_model/variables/variables.index (deflated 63%)
  adding: content/AMAP_alternative_tensorflow_model/variables/variables.data-00000-of-00001 (deflated 46%)
  adding: content/AMAP_alternative_tensorflow_model/saved_model.pb (deflated 88%)
  adding: content/AMAP_alternative_tensorflow_model/assets/ (stored 0%)
  adding: content/AMAP_alternative_tensorflow_model/keras_metadata.pb (deflated 89%)
